In [0]:

import torchvision
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import albumentations as A
import cv2
from torchsummary import summary
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
from albumentations import Compose, Normalize, HorizontalFlip, Cutout, Rotate, RandomCrop, PadIfNeeded
from albumentations.pytorch import ToTensor

In [0]:
#!pip install -U git+https://github.com/albu/albumentations --no-cache-dir


In [2]:
#import Model as Md
import Misclassified_image as MC
from TrainTestandUtils import TrainTestandUtils as Tt
import cyclic_lr as clr
from Albumentations import AlbumentationTransforms
from GradCAM import GradCAM
from LR_range_test import LR_test
from lr_finder import LRFinder
from Utils import *
#import resnet_new as rn

  pip install torch-lr-finder -v --global-option="amp"


In [0]:
from tinyimagenet import TinyImageNetDataSet


In [0]:
#!pip install --no-cache-dir git+https://github.com/albu/albumentations > /dev/null && echo "Albumentation library is successfully installed!"

In [5]:
import albumentations as A
import cv2
channel_means = (0.5, 0.5, 0.5)
channel_stdevs = (0.5, 0.5, 0.5)
train_transform = AlbumentationTransforms([       
                                                                
                                      A.HorizontalFlip(p = 0.7),
                                      A.PadIfNeeded(min_height=70, min_width=70, border_mode=4, value=None, mask_value=None, always_apply=False, p=1.0),
                                      A.GaussNoise(var_limit=(10.0, 50.0), mean=0, always_apply=False, p=0.5),
                                      A.RandomCrop(64, 64, always_apply=False, p=1.0),
                                      A.VerticalFlip(always_apply=False, p=0.5),
                                      A.Rotate(limit=50, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
                                      A.Normalize(mean=channel_means, std=channel_stdevs),
                                      A.Cutout(num_holes=1, max_h_size=32,max_w_size = 32,p=0.7) 
                                       ])
test_transform = AlbumentationTransforms([A.Normalize(mean=channel_means, std=channel_stdevs)])
train_dataset , test_dataset,classes = TinyImageNetDataSet(train_split = 70,test_transforms = test_transform,train_transforms = train_transform)

Images already downloaded...


In [6]:
train_dataset , test_dataset,classes = TinyImageNetDataSet(train_split = 70,test_transforms = test_transform,train_transforms = train_transform)


Images already downloaded...


In [7]:
len(test_dataset)

33000

In [0]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128,
                                          shuffle=True, num_workers=4, pin_memory=True)

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=128,
                                         shuffle=False, num_workers=4)


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

cuda


In [10]:
model = torchvision.models.resnet18(pretrained=False, num_classes=200).to(device)
summary(model, input_size=(3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           9,408
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          36,864
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          36,864
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
       BasicBlock-11           [-1, 64, 16, 16]               0
           Conv2d-12           [-1, 64, 16, 16]          36,864
      BatchNorm2d-13           [-1, 64, 16, 16]             128
             ReLU-14           [-1, 64,

In [0]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR,StepLR

optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9,weight_decay = 0.0001,nesterov = True ) 
criterion = nn.CrossEntropyLoss()
scheduler = OneCycleLR(optimizer, max_lr = 0.01, total_steps=None, epochs=24, steps_per_epoch=1, pct_start=1/3, anneal_strategy='linear', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=10.0,final_div_factor =10)

In [0]:
new_model = Tt(model=model,
               device=device,
               optimizer=optimizer,
               criterion=criterion,
               scheduler=scheduler,
               trainloader=trainloader,
               testloader=testloader,
               epochs=50,
               input_size=(3,64,64),
               classes=classes)

In [0]:
new_model.runModel()

  0%|          | 0/602 [00:00<?, ?it/s]

EPOCH  1


  0%|          | 0/602 [00:00<?, ?it/s]

Test set: Average loss: 0.0382, Accuracy: 1352/33000 (4.10%)

EPOCH  2


  0%|          | 0/602 [00:00<?, ?it/s]

...

Loss=3.239013910293579 Batch_id=601 Accuracy=26.06: 100%|██████████| 602/602 [01:14<00:00,  8.13it/s]
  0%|          | 0/602 [00:00<?, ?it/s]

Test set: Average loss: 0.0254, Accuracy: 8684/33000 (26.32%)

EPOCH  22


Loss=3.091874361038208 Batch_id=601 Accuracy=26.53: 100%|██████████| 602/602 [01:14<00:00,  8.09it/s]
  0%|          | 0/602 [00:00<?, ?it/s]

Test set: Average loss: 0.0255, Accuracy: 8640/33000 (26.18%)

EPOCH  23


Loss=3.272275686264038 Batch_id=548 Accuracy=27.39:  91%|█████████ | 548/602 [01:09<00:05,  9.01it/s] 